In [1]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
import numpy as np
data, target = load_diabetes(return_X_y=True)

## Partial regularization in linear models
This example shows how to perform regularization in **linear models** only for a subset of variables. The choice of the topic is motivated by the following StackOverflow post: https://stats.stackexchange.com/a/307133. Despite the thorough mathematical explanation, you can consider this notebook like a complementary hands-on tutorial. 
 
For this example, the `Diabetes dataset` will be used. A detailed description of this dataset can be found [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html#sklearn.datasets.load_diabetes)

To determine the coefficients with a LASSO model you have to minimize this equation (two independent variables are used for the sake of simplicity):
$$ E = ||\mathbf{y} - \mathbf{X}_1 \boldsymbol{\beta}_1 - \mathbf{X}_2 \boldsymbol{\beta}_2||^2 + \lambda ||\boldsymbol{\beta}||_1$$

As you will notice, regularization is applied equally to both variables.

In [2]:
model_full = Lasso(alpha=0.01)
model_full.fit(data, target)
target_pred = model_full.predict(data)
# Show the RMSE
mean_squared_error(target, target_pred, squared=False)

53.56340262454339

The second case will be partial regularization. This may be handy when dealing with multiple variables and you are pretty sure there are some of them have to be included. Taken directly from the StackOverflow post, I'll show you the equation: 
$$ E = ||\mathbf{y} - \mathbf{X}_1 \boldsymbol{\beta}_1 - \mathbf{X}_2 \boldsymbol{\beta}_2||^2 + \lambda ||\boldsymbol{\beta}_1||_1$$
In our dataset, we choose the blood serum measurements (the last six variables) as $X_1$. Their corresponding coefficients, $\beta_1$, are the ones that will be regularized.
To get their coefficients with Scikit-Learn, everything comes down to the following equation: 
$$
\begin{align*}
 \hat\beta_1 
 & = \arg\min_{\beta_1} \left\{ 0 + \|\left(I-H_2\right)\left(y - X_1\beta_1 \right) \|_2^2 + \lambda \|\beta_1 \|_1 \right\} \\
 & =\arg\min_{\beta_1} \left\{ \|\left(I-H_2\right)y - \left(I-H_2\right)X_1\beta_1 \|_2^2 + \lambda \|\beta_1 \|_1 \right\}
\end{align*}
$$
Here, $X_2$ is assumed to be constrained in a certain space, called $H_2$. The complementary space, defined mathematically as $\left(I-H_2\right)$, will be the space that can be only explained by the coefficient set $\beta_1$. This will be satisfied for any value of $\beta_2$. Now, to arrive to a compatible formula where we use a `Lasso` object of Scikit-Learn, we just need to make the following substitutions.
$$
y^{\prime} = \left(I-H_2\right)y \qquad X^{\prime}_1 = \left(I-H_2\right)X_1
$$
To eventually get:
$$
\hat\beta_1 = \arg\min_{\beta_1} \left\{ \|y^{\prime} - X^{\prime}_1\beta_1 \|_2^2 + \lambda \|\beta_1 \|_1 \right\}
$$

In [3]:
Xreg = data[:,:-6] # X1
Xnot = data[:,-6:] # X2
# This will be our H2
Hnot = Xnot@np.linalg.inv(Xnot.T@Xnot)@Xnot.T
Hnot

array([[ 0.00761648, -0.00579619,  0.00587843, ..., -0.00011387,
         0.00428299, -0.0123322 ],
       [-0.00579619,  0.0182867 , -0.00221394, ..., -0.00270669,
        -0.00196578,  0.01678911],
       [ 0.00587843, -0.00221394,  0.00529319, ..., -0.00026381,
         0.00312404, -0.00744863],
       ...,
       [-0.00011387, -0.00270669, -0.00026381, ...,  0.00695184,
        -0.0037751 , -0.01224746],
       [ 0.00428299, -0.00196578,  0.00312404, ..., -0.0037751 ,
         0.0058978 , -0.00420768],
       [-0.0123322 ,  0.01678911, -0.00744863, ..., -0.01224746,
        -0.00420768,  0.05676557]])

In [5]:
imat = np.eye(data.shape[0]) # Matrix I
Xreg_t = (imat-Hnot)@Xreg # X1'
target_t = (imat-Hnot)@target # y'

model_partial= Lasso(alpha=0.01)
model_partial.fit(Xreg_t, target_t)
target_pred = model_partial.predict(Xreg_t)
#Show the RMSE
mean_squared_error(target_t, target_pred, squared=False)


53.478184817901784

To make a fair comparison we finish by building an ordinary linear model without regularization.

In [6]:
from sklearn.linear_model import LinearRegression

model_OLS = LinearRegression()
model_OLS.fit(data, target)
target_pred = model_OLS.predict(data)
#Show the RMSE
mean_squared_error(target, target_pred, squared=False)

53.47607314274362

Putting the three models in a table, the linear model without regularization is the one with lowest RMSE, followed very closely by the model with partial regularization. The standard LASSO model comes in third place.

| Model | RMSE |
|------|------|
|   OLS  | 53.476 |
| Partial LASSO | 53.478 |
| LASSO | 53.563 |

It needs to be said, this tutorial was just to show how to perform partial regularization. If we were trying to build a good model to predict the target value there would be other steps before getting into building the model.

**Disclaimer:** We use the term partial regularization to express regularization for a subset of variables only in this tutorial. Probably there exists a standard term that has the same meaning.